# Clean up environment
This notebook cleans the environment used in the demo.

In [ ]:
import boto3
session = boto3.Session()
region = session.region_name
sts = boto3.client('sts')
id_info = sts.get_caller_identity()
account = id_info['Account']

## Delete containts of S3 - must be empty before Cloud Formation deleting -

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(f'workshop-timeseries-retail-{account}-cloudtrail')
bucket.objects.all().delete()

In [ ]:
s3.Bucket(f'workshop-timeseries-retail-{account}-source').objects.all().delete()

## Delete StepFunctions

In [ ]:
sfn = boto3.client('stepfunctions')

In [ ]:
sfn.delete_state_machine(
    stateMachineArn=f'arn:aws:states:{region}:{account}:stateMachine:workshop-timeseries-retail'
)

## Delete CloudWatch Events Rule

In [ ]:
cwe = boto3.client('events')

In [ ]:
### confirmation
cwe.list_targets_by_rule(
    Rule='workshop-timeseries-retail',
    #NextToken='string',
    Limit=100
)

In [ ]:
response = cwe.remove_targets(
    Rule='workshop-timeseries-retail',
    Ids=[
        'forecast',
    ]
)

In [ ]:
response = cwe.delete_rule(
    Name='workshop-timeseries-retail'
)

## Delete Amazon Foreast components

In [ ]:
import time

In [ ]:
forecast = boto3.client('forecast')

In [ ]:
### exportjob
forecast.delete_forecast_export_job(
    ForecastExportJobArn=f'arn:aws:forecast:{region}:{account}:forecast-export-job/tr_target_add_20091201_20101209/tr_target_add_20091201_20101209'
)

while 1:
    print('Waiting for deleting...')
    time.sleep(10)
    try:
        response = forecast.describe_forecast_export_job(
            ForecastExportJobArn=f'arn:aws:forecast:{region}:{account}:forecast-export-job/tr_target_add_20091201_20101209/tr_target_add_20091201_20101209'
        )
    except:
        print('Deleted.')
        break

In [ ]:
### forecast
forecast.delete_forecast(
    ForecastArn=f'arn:aws:forecast:{region}:{account}:forecast/tr_target_add_20091201_20101209'
)

while 1:
    print('Waiting for deleting...')
    time.sleep(10)
    try:
        response = forecast.describe_forecast(
            ForecastArn=f'arn:aws:forecast:{region}:{account}:forecast/tr_target_add_20091201_20101209'
        )
    except:
        print('Deleted.')
        break

In [ ]:
### predictor
forecast.delete_predictor(
    PredictorArn=f'arn:aws:forecast:{region}:{account}:predictor/tr_target_add_20091201_20101209'
)

while 1:
    print('Waiting for deleting...')
    time.sleep(10)
    try:
        response = forecast.describe_predictor(
            PredictorArn=f'arn:aws:forecast:{region}:{account}:predictor/tr_target_add_20091201_20101209'
        )
    except:
        print('Deleted.')
        break

In [ ]:
### dataset import job
forecast.delete_dataset_import_job(
    DatasetImportJobArn=f'arn:aws:forecast:{region}:{account}:dataset-import-job/workshop_timeseries_retail_target/tr_target_add_20091201_20101209'
)

while 1:
    print('Waiting for deleting...')
    time.sleep(10)
    try:
        response = forecast.describe_dataset_import_job(
            DatasetImportJobArn=f'arn:aws:forecast:{region}:{account}:dataset-import-job/workshop_timeseries_retail_target/tr_target_add_20091201_20101209'
        )
    except:
        print('Deleted.')
        break

In [ ]:
### dataset
forecast.delete_dataset(
    DatasetArn=f'arn:aws:forecast:{region}:{account}:dataset/workshop_timeseries_retail_target'
)

while 1:
    print('Waiting for deleting...')
    time.sleep(10)
    try:
        response = forecast.describe_dataset(
            DatasetArn=f'arn:aws:forecast:{region}:{account}:dataset/workshop_timeseries_retail_target'
        )
    except:
        print('Deleted.')
        break

In [ ]:
### datasetgroup
forecast.delete_dataset_group(
    DatasetGroupArn=f'arn:aws:forecast:{region}:{account}:dataset-group/workshop_timeseries_retail'
)

while 1:
    print('Waiting for deleting...')
    time.sleep(10)
    try:
        response = forecast.describe_dataset_group(
            DatasetGroupArn=f'arn:aws:forecast:{region}:{account}:dataset-group/workshop_timeseries_retail'
        )
    except:
        print('Deleted.')
        break

## Delete S3 bucket for deployment

In [ ]:
s3.Bucket(f'workshop-timeseries-retail-{account}-deploy').objects.all().delete()

In [ ]:
s3.Bucket(f'workshop-timeseries-retail-{account}-deploy').delete()

## Delete CloudFormation Stack
The SageMaker notebook instance running this notebook will also be deleted.

In [ ]:
cf = boto3.client('cloudformation')
res = cf.delete_stack(
    StackName=('workshop-timeseries-retail')
)

## End of Workshop